# Sorting out basic design issues

## Start Focus on Bidder Side - Make bidding decisions

In [9]:
import numpy as np
import random

class User:
    def __init__(self):
        self.probability2 = random.uniform(0, 1)

    def show_ad(self):
        return  np.random.choice((True, False), p = [self.probability2, 1-self.probability2])

class Auction:
    
    auction_history = [] # for the auction exchange only
    balances = {}
    
    def __init__(self, users, bidders):
        self.users = users
        self.bidders = bidders
        
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        print("*"*100, '\n')
        print('Beginning balances : ',self.balances, '\n','-'*100)
        
        
    def execute_round(self):
        
        ##### if the -ve balance of ALL bidders is < -1000, then break
        
        all_invalid_bidders = any(x > -1000 for x in self.balances.values())
        
        while not all_invalid_bidders:
            print("no bidders are qualifed as all balances are below -1000")
            break
        
        else:
            # print('starting balances2: ', self.balances2, '\n','-'*100)
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            # 1. SELECT USER (from the pool of users) ----------------------------------

            chosen_user = random.randint(0, len(self.users)-1)
            print('Selected User: ', id(chosen_user), '\n','-'*100) 

            # 2. COLLECT BIDS (send to all bidders) --------------------------------------------
            for bidder in self.bidders:
                #if self.balances[bidder] > -1000:
                #balances = {bidder: 0 for bidder in bidders} ########### NEW LINE
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            print("Current bidding Round: ", bidding_round, '\n','-'*100)

            # 3. DETERMINE WINNING BID --------------------------------------------
            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))


    #        print('bids list', bids_list, '\n','-'*100)

            ###### SORT THE Bids List based on price ##########################################
            sorted_list = sorted(bids_list, key=lambda t:t[1])
            print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis - creating a copy with id(bidder) --------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
    #        print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ####### Select second highest price ###############################################
            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ######## Determine if there are more than 1 bidders with same price ######################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            print('Winning Bidder (FINAL): ', id(winning_bidder), '\n','-'*100)
            print('Winning Price (FINAL): ', winning_price, '\n','-'*100)

            ############################################################################################
            # 4. Validate USER CLICK after SHOW AD  --------------------------------------------
            ad_result = self.users[chosen_user].show_ad()
            print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            # 5.NOTIFY BIDDER & UPDATE BALANCES --------------------------------------------

            print("Bidding Outcomes", '\n','-'*100)
            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1
                    # balances[bidder] -= winning_price
                    # balances[bidder] += 1                

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price
                    # balances[bidder] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)
                # print('balances at end of bidding round', balances,'\n','-'*100)
        
class Bidder:
    
    #### maintain account balance here as well?

    # BIDDER CREATION -------------- initiated by GAME runner / Developer ------------------------
    
    def __init__(self, num_users, num_rounds):

        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0

        self.bid_y_n = True
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins_no_click = {i: 0 for i in range(num_users)} # if bidder wins
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}
        
        #Transaction records
        self.bid_participate_history = [] # maintains bidderid, userid, bidding round, bid price
        self.bid_win_history = [] # won bid
        self.bid_win_user_click_history = [] # won bid and user clicked
        self.bid_lost_history = [] # lost bid, but know the user and price
        
        
        ########### New tables
        self.bid_win_click_no = []
        self.bid_win_click_yes = []
        self.bid_lost_list = []
        self.bid_360 = []
        
        ############
        #? Winning bid price / user (because you get notified of winning bids) -- 
        ###### you know the user and winning bid (just not the click result)
        
    # SUBMIT BID  ------------ initiated by auction ------------------------------------
    
    def bid(self, user_id):
        self.bidding_round += 1
        #self.bid_participate = 0
        self.user_id = user_id
        default_bid_price = random.randint(1000, 10000)/2000 #random.uniform(0, 1)
        # bid_price = self.default_bid_price + (self.bidding_round)*0.1
        bid_price = default_bid_price
        #bid_price = 0.5
        self.bid_participate[self.user_id] = (self.bid_participate).get(self.user_id, 0)+1
        self.bid_participate_history.append([id(self), self.user_id, self.bidding_round, bid_price])
        return bid_price
    
#    def bid_strategy(self):
        
        #self.bid_360.extend(self.bid_win_history, self.bid_lost_history)
        self.bid_360 = [item for sublst in zip(self.bid_win_history,  self.bid_lost_history) for item in sublst]

        # NOTIFIED OF BID OUTCOME ------------ initiated by auction ------------------------------------
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        
        if auction_winner == True and clicked == True:
            self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
            #self.bid_win_user_click_history.append([self.user_id, self.bidding_round])
            
            self.bid_win_click_yes.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

        elif auction_winner == True and clicked == False:
            #self.bid_participate_history.append([self.user_id, bid_y_n_won+1])
            self.bid_wins_no_click[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
            # self.bid_win_history.append([self.user_id, bid_y_n_won+1])

#            self.bid_win_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            self.bid_win_click_no.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
        
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1

#            self.bid_lost_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

            self.bid_lost_list.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

            self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            # NOTIFICATION: bidder.notify(False, winning_price, None)
            
        
        
        
        
        
        
#         if auction_winner:
#             #self.bid_participate_history.append([self.user_id, bid_y_n_won+1])
#             self.bid_wins_no_clilck[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
#             # self.bid_win_history.append([self.user_id, bid_y_n_won+1])
#             self.bid_win_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
#             self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
#             if clicked:
#                 self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
#                 #self.bid_win_user_click_history.append([self.user_id, self.bidding_round])
#                 self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
#         else:
#             self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1
#             self.bid_lost_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
#             self.bid_360.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
#             # NOTIFICATION: bidder.notify(False, winning_price, None)
            pass

In [10]:
b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3])
#auction = Auction( [User(), User()], [b1, b2, b3])

auction.execute_round()
# auction.execute_round()
# auction.execute_round()
# auction.execute_round()

**************************************************************************************************** 

Beginning balances :  {140706106992864: 0, 140706106990704: 0, 140706095165600: 0} 
 ----------------------------------------------------------------------------------------------------
Selected User:  140706081466704 
 ----------------------------------------------------------------------------------------------------
Current bidding Round:  1 
 ----------------------------------------------------------------------------------------------------
Sorted bids list [(140706106992864, 1.0545), (140706095165600, 1.324), (140706106990704, 3.916)] 
 ----------------------------------------------------------------------------------------------------
Winning Bidder (FINAL):  140706106990704 
 ----------------------------------------------------------------------------------------------------
Winning Price (FINAL):  1.324 
 -----------------------------------------------------------------------

In [12]:
print('Bidding Rounds', b1.bidding_round)
print('Bid Wins User Clicked', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
print('Bid Wins User NO Clicked', {k: v for k, v in sorted(b1.bid_wins_no_click.items(), key=lambda item: item[0])})
print('Bids Lost', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])})


Bidding Rounds 101
Bid Wins User Clicked {0: 1, 1: 1, 3: 1, 5: 3, 7: 3, 8: 1, 9: 2}
Bid Wins User NO Clicked {0: 2, 1: 3, 2: 2, 3: 1, 5: 1, 6: 1, 7: 1, 8: 4, 9: 1}
Bids Lost {0: 7, 1: 3, 2: 9, 3: 10, 4: 4, 5: 11, 6: 5, 7: 7, 8: 12, 9: 5}


In [8]:
### DNU

print('Bidding Rounds', b1.bidding_round)
#print('Bid Participate', b1.bid_participate)
print('Bid Participate Sorted', {k: v for k, v in sorted(b1.bid_participate.items(), key=lambda item: item[0])})
#print('Bid Wins', b1.bid_wins)
print('Bidding Wins Sorted', {k: v for k, v in sorted(b1.bid_wins.items(), key=lambda item: item[0])})
#print('User Clicks', b1.bid_win_user_clicks)
print('User Clicks Sorted', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
#print('Bids Lost', b1.bid_lost)
print('Bids Lost Sorted', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])}) 

Bidding Rounds 101
Bid Participate Sorted {0: 6, 1: 14, 2: 10, 3: 12, 4: 12, 5: 7, 6: 4, 7: 10, 8: 15, 9: 11}


AttributeError: 'Bidder' object has no attribute 'bid_wins'

In [13]:
print('Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
b1.bid_win_click_yes # 1.bidding_round, 2. User_id, 3. auction_winner 4.self.price  5. self.clicked

Bid Win - Click YES: Count 12


[[13, 5, True, 2.359, True],
 [18, 7, True, 3.9835, True],
 [19, 9, True, 2.187, True],
 [22, 7, True, 3.843, True],
 [27, 0, True, 4.391, True],
 [28, 1, True, 4.7665, True],
 [32, 5, True, 0.619, True],
 [50, 8, True, 3.264, True],
 [60, 5, True, 3.6035, True],
 [77, 3, True, 2.9235, True],
 [88, 7, True, 3.996, True],
 [90, 9, True, 3.0495, True]]

In [14]:
print('Bid Win - Click NO: Count',len(b1.bid_win_click_no))
b1.bid_win_click_no # 1.bidding_round, 2. User_id, 3. auction_winner 4.self.price  5. self.clicked

Bid Win - Click NO: Count 16


[[9, 2, True, 3.734, False],
 [11, 2, True, 0.6645, False],
 [16, 0, True, 2.108, False],
 [17, 8, True, 3.047, False],
 [20, 8, True, 2.9105, False],
 [37, 7, True, 4.1835, False],
 [38, 8, True, 1.793, False],
 [40, 9, True, 2.49, False],
 [49, 1, True, 1.568, False],
 [59, 6, True, 3.098, False],
 [62, 5, True, 2.2285, False],
 [67, 1, True, 3.6215, False],
 [72, 3, True, 3.1845, False],
 [73, 1, True, 3.3415, False],
 [84, 8, True, 0.7885, False],
 [85, 0, True, 1.907, False]]

In [15]:
print('Bid LOST: Count',len(b1.bid_lost_list))
b1.bid_lost_list # 1.bidding_round, 2. User_id, 3. auction_winner 4.self.price  5. self.clicked

Bid LOST: Count 73


[[1, 2, False, 1.324, None],
 [2, 3, False, 3.232, None],
 [3, 3, False, 3.409, None],
 [4, 0, False, 2.3705, None],
 [5, 8, False, 2.4775, None],
 [6, 9, False, 2.3805, None],
 [7, 7, False, 1.9025, None],
 [8, 3, False, 3.0485, None],
 [10, 8, False, 4.446, None],
 [12, 8, False, 1.511, None],
 [14, 3, False, 2.8415, None],
 [15, 2, False, 0.9605, None],
 [21, 2, False, 1.4575, None],
 [23, 4, False, 1.346, None],
 [24, 0, False, 0.988, None],
 [25, 5, False, 1.984, None],
 [26, 6, False, 2.9935, None],
 [29, 6, False, 4.3085, None],
 [30, 5, False, 4.7185, None],
 [31, 7, False, 1.559, None],
 [33, 3, False, 4.2315, None],
 [34, 1, False, 1.179, None],
 [35, 6, False, 3.856, None],
 [36, 8, False, 1.453, None],
 [39, 5, False, 2.91, None],
 [41, 4, False, 4.0105, None],
 [42, 1, False, 2.001, None],
 [43, 7, False, 1.5795, None],
 [44, 5, False, 2.7735, None],
 [45, 7, False, 3.0465, None],
 [46, 5, False, 2.4475, None],
 [47, 8, False, 2.6015, None],
 [48, 6, False, 3.067, None],
 

In [11]:
[auction.execute_round() for i in range(100)]

Selected User:  140706081466736 
 ----------------------------------------------------------------------------------------------------
Current bidding Round:  2 
 ----------------------------------------------------------------------------------------------------
Sorted bids list [(140706095165600, 1.204), (140706106992864, 3.232), (140706106990704, 4.2575)] 
 ----------------------------------------------------------------------------------------------------
Winning Bidder (FINAL):  140706106990704 
 ----------------------------------------------------------------------------------------------------
Winning Price (FINAL):  3.232 
 ----------------------------------------------------------------------------------------------------
User Clicked on Ad?:  True 
 ----------------------------------------------------------------------------------------------------
Bidding Outcomes 
 ----------------------------------------------------------------------------------------------------
140706106

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [53]:
auction.balances

{140276743991200: -1000.3605,
 140276743990432: -1000.5714999999998,
 140276743982816: -1000.3000000000001}

In [7]:
b1.bidding_round

101

In [14]:
b1.bid_participate_history #bidderid, userid, bidding round, bid price

[[140467995445472, 5, 1, 2.633]]

In [ ]:
b2.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
b3.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
[auction.execute_round() for i in range(10)]

In [ ]:
b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()], [b1, b2, b3])
auction.execute_round()

In [ ]:
print('b1 bid is: ', b1.bid(0x7fb6581c8ca0))
print('b2 bid is: ', b2.bid(0x7fb6581c8ca0))
print('b3 bid is: ', b3.bid(0x7fb6581c8ca0))

In [ ]:
u1 = User()
id(u1)

In [ ]:
user_wins = {i: 0 for i in range(10)}
user_wins

In [17]:
bids_list = [(1,2), (7,8), (5,6),(3,4) ]

In [18]:
sorted_list = sorted(bids_list, key=lambda t:t[1])
sorted_list

[(1, 2), (3, 4), (5, 6), (7, 8)]

In [19]:
winning_price = (sorted_list)[-2][1]
winning_price

6

In [21]:
winning_bid = (sorted_list)[-1][0]
winning_bid

7

In [22]:
list2 = [(1,2)]
sorted_list = sorted(list2, key=lambda t:t[1])
sorted_list

[(1, 2)]

In [23]:
winning_price = (sorted_list)[0][-1]

In [24]:
winning_price

2

In [26]:
winning_bid = (sorted_list)[0][0]
winning_bid

1